# 📈 Forecasting Weekly Demand using Prophet

This notebook uses the fictional retail dataset to forecast weekly product demand using Facebook Prophet.

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de gráficos
plt.style.use('seaborn-whitegrid')
sns.set()


In [ ]:
# Load dataset
df = pd.read_csv('../data/demanda_retail_ficticio.csv')
df['fecha'] = pd.to_datetime(df['fecha'])
df.head()

In [ ]:
# Filter a single SKU and warehouse to model
sku = 'SKU_A'
almacen = 'CEDIS_Norte'

df_sku = df[(df['sku'] == sku) & (df['almacen'] == almacen)].copy()
df_sku = df_sku[['fecha', 'unidades_vendidas']].rename(columns={'fecha': 'ds', 'unidades_vendidas': 'y'})
df_sku.head()

In [ ]:
# Create and fit the Prophet model
model = Prophet(weekly_seasonality=True)
model.fit(df_sku)

# Create future dataframe (6 weeks ahead)
future = model.make_future_dataframe(periods=6, freq='W')
forecast = model.predict(future)

# Mostrar forecast
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10)

In [ ]:
# Plot forecast
fig1 = model.plot(forecast)
plt.title(f"Forecast for {sku} at {almacen}")
plt.show()

# Plot components
fig2 = model.plot_components(forecast)
plt.show()

In [ ]:
# Calcular MAPE en datos históricos
df_merged = df_sku.merge(forecast[['ds', 'yhat']], on='ds', how='left')
df_merged.dropna(inplace=True)
df_merged['error'] = abs(df_merged['y'] - df_merged['yhat'])
df_merged['error_pct'] = df_merged['error'] / df_merged['y']
mape = df_merged['error_pct'].mean() * 100
print(f"MAPE: {mape:.2f}%")